# Create cloze-question aligned dataset from squad

In [39]:
with open("/home/mroemmele/CoreNLP/others/reading_comprehension/v1/train-v1.1.json") as f:
    squad_data = json.load(f)

In [40]:
squad_data['data'][0]['paragraphs']

[{'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
  'qas': [{'answers': [{'answer_start': 515,
      'text': 'Saint Bernadette Soubirous'}],
    'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
    'id': '5733be284776f41900661182'},
   {'answers': [{'answer_start': 188, 'text': 'a copper statue of 

In [ ]:
clozes = []
questions = []
# answers = []
for article in squad_data['data']:
    for paragraph in article['paragraphs']:
        context = paragraph['context']
        for qa in paragraph['qas']:
            answer = qa['answers']
            question = qa['question']
        context[answer['answer_start']:answer['answer_start'] + len(answer['text'])]
        p_questions, p_answers  = zip(*[(qa['question'], qa['answers'][0]['text'])
                               for qa in paragraph['qas']])
        questions.extend(p_questions)
        answers.extend(p_answers)

# Count entity types in squad answers

In [1]:
import pickle


In [43]:
with open('answers.pkl', 'rb') as f:
    answers = pickle.load(f)

In [48]:
with open('contexts.pkl', 'rb') as f:
    contexts = pickle.load(f)

In [49]:
contexts[:2]

[[('Architecturally', 'O'),
  (',', 'O'),
  ('the', 'O'),
  ('school', 'O'),
  ('has', 'O'),
  ('a', 'O'),
  ('Catholic', 'NORP'),
  ('character', 'O'),
  ('.', 'O'),
  ('Atop', 'O'),
  ('the', 'O'),
  ('Main Building', 'LOCATION'),
  ("'s", 'O'),
  ('gold', 'O'),
  ('dome', 'O'),
  ('is', 'O'),
  ('a', 'O'),
  ('golden', 'O'),
  ('statue', 'O'),
  ('of', 'O'),
  ('the', 'O'),
  ('Virgin Mary', 'PERSON'),
  ('.', 'O'),
  ('Immediately', 'O'),
  ('in', 'O'),
  ('front', 'O'),
  ('of', 'O'),
  ('the', 'O'),
  ('Main Building', 'LOCATION'),
  ('and', 'O'),
  ('facing', 'O'),
  ('it', 'O'),
  (',', 'O'),
  ('is', 'O'),
  ('a', 'O'),
  ('copper', 'O'),
  ('statue', 'O'),
  ('of', 'O'),
  ('Christ', 'PERSON'),
  ('with', 'O'),
  ('arms', 'O'),
  ('upraised', 'O'),
  ('with', 'O'),
  ('the', 'O'),
  ('legend', 'O'),
  ('"', 'O'),
  ('Venite Ad Me Omnes', 'LOCATION'),
  ('"', 'O'),
  ('.', 'O'),
  ('Next', 'O'),
  ('to', 'O'),
  ('the', 'O'),
  ('Main Building', 'LOCATION'),
  ('is', 'O'),
  (

In [7]:
answer_type_counts = {}
for answer in answers:
    answer_type = tuple([token[-1] for token in answer])
    if answer_type not in answer_type_counts:
        answer_type_counts[answer_type] = 0
    answer_type_counts[answer_type] += 1

In [45]:
len(answer_type_counts)

4022

In [23]:
from collections import defaultdict

meta_answer_type_counts = defaultdict(int)
for answer_type, count in answer_type_counts.items():
    if len(answer_type) == 1 and answer_type[0] != 'O':
        meta_answer_type_counts[answer_type[0] + "_ONLY"] += count
    else:
        entity_set = set([token_type for token_type in answer_type])
        if len(entity_set) == 1 and 'O' in entity_set:
            meta_answer_type_counts['NO_ENTITY'] += count
        entity_set.discard('O')
        for entity in list(entity_set):
            meta_answer_type_counts[entity + '_COMBO'] += count

In [36]:
import pandas

pandas.DataFrame(sorted(meta_answer_type_counts.items()), index=None, columns=['EntityType', '#Answers']).to_csv('squad_answer_types.csv', index=False)

In [38]:
sum([count for entity_type, count in meta_answer_type_counts.items() if entity_type.endswith('ONLY')])

34451